In [1]:
import numpy as np
from optimal_levee_bandit import (
    load_cost_curves,
    prune_candidates,
    run_bandit_from_candidates,
)

damage_file = "Damage_cost_curves.tab"
protection_file = "Protection_cost_curves_high_estimate.tab"
city = "Halmstad"
years_range = (2025, 2100)

# --- 1) Load cost curves ---
heights, damage_costs, protection_costs = load_cost_curves(
    damage_file, protection_file, city
)

# --- 2) Prune candidates ---
start_year, end_year = years_range
n_years = end_year - start_year + 1

candidate_indices = prune_candidates(
    heights=heights,
    protection_costs=protection_costs,
    damage_costs=damage_costs,
    n_years=n_years,
)

print(f"Pruned from {len(heights)} to {len(candidate_indices)} candidate heights.")
print("Candidate heights (m):", heights[candidate_indices])



Pruned from 25 to 25 candidate heights.
Candidate heights (m): [ 0.   0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5  6.   6.5
  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5 12. ]


In [2]:
# --- 3) Load predictive posterior + future MSL paths from NPZ ---
pp = np.load("pp_inputs_halmsdad_pp_mixture_2025_2100.npz")

# Posterior draws (flattened)
eta0_s   = pp["eta0"].reshape(-1)
eta1_s   = pp["eta1"].reshape(-1)
alpha0_s = pp["alpha0"].reshape(-1)
xi_s     = pp["xi"].reshape(-1)
u_cm     = float(pp["u"])  # threshold in cm, if you want to store/pass it

posterior_params = {
    "eta0": eta0_s,
    "eta1": eta1_s,
    "alpha0": alpha0_s,
    "xi": xi_s,
    # Optional: if you want run_bandit_from_candidates to pick it up:
    "u": u_cm,
}

# Future MSL information (already in cm)
years_future   = pp["years_future"]      # shape (T_future,)
X_future_paths = pp["X_future_paths"]    # shape (M_pred, T_future), in cm

# --- 4) Run the bandit on the pruned candidate set ---
rng = np.random.default_rng(42)

best_height, history = run_bandit_from_candidates(
    heights=heights,
    damage_costs=damage_costs,
    protection_costs=protection_costs,
    candidate_indices=candidate_indices,
    years_all=years_future,
    X_pred_paths_cm=X_future_paths,
    posterior_params=posterior_params,
    years_range=years_range,
    delta=0.05,
    max_rounds=100000,
    rng=rng,
    verbose=True,
)

best_height


Round 1000: best height 2.00 m, mean cost 269.673, r=28349.362, separated=False
Round 2000: best height 2.00 m, mean cost 271.483, r=20705.403, separated=False
Round 3000: best height 2.00 m, mean cost 267.833, r=17213.065, separated=False
Round 4000: best height 2.00 m, mean cost 266.639, r=15092.864, separated=False
Round 5000: best height 2.00 m, mean cost 267.499, r=13627.056, separated=False
Round 6000: best height 2.00 m, mean cost 269.154, r=12534.105, separated=False
Round 7000: best height 2.00 m, mean cost 267.402, r=11677.682, separated=False
Round 8000: best height 2.00 m, mean cost 266.897, r=10982.564, separated=False
Round 9000: best height 2.00 m, mean cost 266.412, r=10403.357, separated=False
Round 10000: best height 2.00 m, mean cost 264.953, r=9910.801, separated=False
Round 11000: best height 2.00 m, mean cost 264.059, r=9485.068, separated=False
Round 12000: best height 2.00 m, mean cost 264.339, r=9112.171, separated=False
Round 13000: best height 2.00 m, mean co

KeyboardInterrupt: 

In [1]:
import numpy as np
from optimal_levee_bandit import (
    load_cost_curves,
    prune_candidates,
    run_bandit_from_candidates_eff,  # our new efficient function
)

damage_file = "Damage_cost_curves.tab"
protection_file = "Protection_cost_curves_high_estimate.tab"
city = "Halmstad"
years_range = (2025, 2100)

# 1. Load cost curves
heights, damage_costs, protection_costs = load_cost_curves(
    damage_file, protection_file, city
)

# 2. Prune deterministically dominated heights
start_year, end_year = years_range
n_years = end_year - start_year + 1
candidate_indices = prune_candidates(
    heights=heights,
    protection_costs=protection_costs,
    damage_costs=damage_costs,
    n_years=n_years,
)
print(f"Pruned from {len(heights)} to {len(candidate_indices)} candidate heights.")
print("Candidate heights (m):", heights[candidate_indices])


Pruned from 25 to 25 candidate heights.
Candidate heights (m): [ 0.   0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5  6.   6.5
  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5 12. ]


In [ ]:

# 3. Load predictive posterior and future MSL paths from your NPZ
pp = np.load("pp_inputs_halmsdad_pp_mixture_2025_2100.npz")
posterior_params = {
    "eta0": pp["eta0"],
    "eta1": pp["eta1"],
    "alpha0": pp["alpha0"],
    "xi": pp["xi"],
    "u": float(pp["u"]),  # optional threshold (cm)
}
years_future   = pp["years_future"]
X_future_paths = pp["X_future_paths"]  # already in cm

# 4. Run the efficient bandit
rng = np.random.default_rng(42)
best_height, history = run_bandit_from_candidates_eff(
    heights=heights,
    damage_costs=damage_costs,
    protection_costs=protection_costs,
    candidate_indices=candidate_indices,
    years_all=years_future,
    X_pred_paths_cm=X_future_paths,
    posterior_params=posterior_params,
    years_range=years_range,
    delta=0.05,
    max_rounds=10000000,
    rng=rng,
    verbose=True,
)

print(f"Selected design height: {best_height:.2f} m")



Round 100000 – mean total cost per candidate height:
  height = 0.00 m, mean total cost = 2041.578
  height = 0.50 m, mean total cost = 2060.106
  height = 1.00 m, mean total cost = 2028.238
  height = 1.50 m, mean total cost = 1458.832
  height = 2.00 m, mean total cost = 922.871
  height = 2.50 m, mean total cost = 710.956
  height = 3.00 m, mean total cost = 674.724
  height = 3.50 m, mean total cost = 717.642
  height = 4.00 m, mean total cost = 814.181
  height = 4.50 m, mean total cost = 944.380
  height = 5.00 m, mean total cost = 1100.913
  height = 5.50 m, mean total cost = 1288.429
  height = 6.00 m, mean total cost = 1499.487
  height = 6.50 m, mean total cost = 1744.152
  height = 7.00 m, mean total cost = 2003.489
  height = 7.50 m, mean total cost = 2276.783
  height = 8.00 m, mean total cost = 2563.104
  height = 8.50 m, mean total cost = 2874.243
  height = 9.00 m, mean total cost = 3195.570
  height = 9.50 m, mean total cost = 3535.763
  height = 10.00 m, mean total c

In [14]:
import math
import numpy as np
delta=0.05
# --- Helper: required samples for a given assumed gap ---
def required_samples_for_gap(
    gap: float,
    D_max_total: float,
    num_cands: int,
    delta: float,
    S_max: int = 10**16,
) -> int:
    """
    Solve for S such that 2 * r(S) <= gap, where
    r(S) = D_max_total * sqrt( log(2 * K * S * (S + 1) / delta) / (2 S) ).
    """
    target = gap / 2.0  # need r(S) <= gap/2

    def radius(S: int) -> float:
        log_term = math.log(
            (2.0 * num_cands * S * (S + 1)) / max(delta, 1e-16)
        )
        return D_max_total * math.sqrt(log_term / (2.0 * S))

    # If S=1 already enough (unlikely), return 1
    if radius(1) <= target:
        return 1

    # Exponential search to bracket S
    low, high = 1, 1
    while high < S_max and radius(high) > target:
        low = high
        high *= 2

    if high >= S_max and radius(high) > target:
        # Could not find S within S_max
        return S_max

    # Binary search between low and high
    while low + 1 < high:
        mid = (low + high) // 2
        if radius(mid) <= target:
            high = mid
        else:
            low = mid

    return high


# --- Compute D_max_total and num_cands from your existing objects ---

# damage_costs: full damage curve (million EUR)
# years_range: (start_year, end_year)
# candidate_indices: indices of surviving heights
# delta: your bandit delta

start_year, end_year = years_range
n_years = end_year - start_year + 1
d_max = float(np.max(damage_costs))
D_max_total = n_years * d_max
num_cands = len(candidate_indices)

# --- Assume the gap between best & second best is about 30 ---
gap_assumed = 30.0

S_req = required_samples_for_gap(
    gap=gap_assumed,
    D_max_total=D_max_total,
    num_cands=num_cands,
    delta=delta,
)

print(f"Assumed gap = {gap_assumed:.1f}")
print(f"Approx. total samples needed for separation: {S_req}")


Assumed gap = 30.0
Approx. total samples needed for separation: 9095155553


In [3]:
((9095155553/10000000)*(9*60))/(3600*24)

5.684472220625